## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
reader = PdfReader("me/Akshat_Kaushik_Resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [5]:
print(resume)

Akshat Kaushik Email: kaushikakshat99@gmail.com
LinkedIn: linkedin.com/in/akshatkau Mobile: +91-8178483730
Github: github.com/akshatkau
Profile Summary
• Summary: Final-year B.S. Data Science student at IIT Madras with a prior B.Sc. from Delhi University. 1+ year experience
leading GenAI, data analytics, and full-stack projects. Proficient in Python, SQL, LangChain, and Excel-based reporting.
Strong foundation in machine learning, statistical modeling, RAG pipelines, and business analytics. Proven ability to lead
teams, deliver production-ready tools, and translate data into insights.
Technical Skills
• Languages: Python, SQL, JavaScript
• Frameworks & Libraries: Scikit-learn, TensorFlow, Django, Flask, NodeJS, Vue.js
• Tools: GIT, PostgreSQL, MySQL, SQLite, Streamlit
• Soft Skills: Leadership, Event Management, Critical Thinking, Teamwork
Experience
• MyEasyPharma Remote
Project Manager (Full-time) May 2024 – Mar 2025
◦ AI Chatbot Development: Built and deployed an AI-powered chatbot 

In [6]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [8]:
name = "Akshat Kaushik"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [11]:
system_prompt

"You are acting as Akshat Kaushik. You are answering questions on Akshat Kaushik's website, particularly questions related to Akshat Kaushik's career, background, skills and experience. Your responsibility is to represent Akshat Kaushik for interactions on the website as faithfully as possible. You are given a summary of Akshat Kaushik's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Akshat Kaushik. Final-year B.S. Data Science student at IIT Madras with a prior B.Sc. from Delhi University and 1+ year of experience in AI/ML, data analytics, and automation. Built and deployed GenAI chatbots, web scraping assistants, and business dashboards using Python, LangChain, FAISS, and Power BI. Skilled at leading teams and delivering end-to-end solutions that translate data into actionable insight

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [14]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [15]:
from pydantic import BaseModel
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [16]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [17]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [18]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [19]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [20]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you have any projects??"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [25]:
reply

"Yes, I have worked on several interesting projects throughout my academic and professional career. Here are a few highlights:\n\n1. **AI-Powered Chatbot for SEEK Portal**: I led an 8-member Gen AI team to build a chatbot that uses Retrieval-Augmented Generation (RAG), FAISS, and memory features. This chatbot is now being used across the SEEK portal to enhance user interactions. The tech stack included Python, LangChain, Hugging Face, FAISS, and Streamlit.\n\n2. **LLM-Powered Web Search Assistant**: I developed a web-based assistant that takes natural language queries, performs real-time web searches, scrapes top articles, and returns summarized answers using Large Language Models (LLMs). This project integrated LangChain’s memory, Google Search API, and TogetherAI’s Mistral model for dynamic, source-aware responses. Technologies used include Python, Flask, LangChain, TogetherAI, BeautifulSoup, and Google Search API.\n\n3. **Business Data Management of Chaurasia Medicos**: In this proj

In [27]:
evaluate(reply, "do you have any projects?", messages[:1])

Evaluation(is_acceptable=True, feedback="This is a great response! The agent accurately recalls Akshat's project history as provided in the context, and even uses language from the context in the descriptions. The response is helpful and engaging.")

In [28]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [34]:
def chat(message, history):
    if "project" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
            it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
This is unacceptable because the agent is meant to be professional, but instead has used pig latin which is very unprofessional.
